<a href="https://colab.research.google.com/github/19K61A1230/cyberbullying1/blob/main/genetic_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install genetic_selection


In [ ]:
pip install sklearn-genetic


In [ ]:
import joblib


In [ ]:
# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

import time

# Load the dataset
data = pd.read_csv('/content/public_data_labeled 2.csv')

# Preprocessing the data
data['full_text'] = data['full_text'].str.lower() # Convert to lowercase
data['full_text'] = data['full_text'].str.replace('[^\w\s]','') # Remove special characters
data['full_text'] = data['full_text'].str.replace('\d+', '') # Remove digits
data['full_text'] = data['full_text'].str.strip() # Remove leading/trailing whitespaces

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['full_text'], data['label'], test_size=0.2, random_state=42)

# Vectorizing the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

# Training the model using SVM
svm_model = LinearSVC()
start_time = time.time()
svm_model.fit(X_train, y_train)
end_time = time.time()

# Predicting on the test set
y_pred = svm_model.predict(X_test)

# Evaluating the model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('Time taken:', end_time - start_time, 'seconds')

# Saving the SVM model
joblib.dump(svm_model, 'svm_model.joblib')


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from genetic_selection import GeneticSelectionCV
import joblib

# Load the dataset
df = pd.read_csv('/content/public_data_labeled 2.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], df['label'], test_size=0.2, random_state=42)

# Create a CountVectorizer object to convert text to numerical features
vectorizer = CountVectorizer()

# Convert the training data to a matrix of token counts
X_train_counts = vectorizer.fit_transform(X_train)

# Initialize the LinearSVC model
clf = LinearSVC(tol=5000)

# Define the genetic algorithm object to perform feature selection
selector = GeneticSelectionCV(clf,
                              cv=5,
                              verbose=1,
                              scoring="accuracy",
                              n_population=15,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=4,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=2,
                              caching=True,
                              n_jobs=-1)

# Fit the genetic algorithm to the training data and select the best features
selector.fit(X_train_counts, y_train)

# Use the selected features to transform the training data
X_train_selected = selector.transform(X_train_counts)

# Fit the SVM model on the selected features
clf.fit(X_train_selected, y_train)

# Convert the testing data to a matrix of token counts and transform using the selected features
X_test_counts = vectorizer.transform(X_test)
X_test_selected = selector.transform(X_test_counts)

# Make predictions on the testing data
y_pred = clf.predict(X_test_selected)

# Calculate accuracy, precision and recall of the model
accuracy = accuracy_score(y_test, y_pred)



print("Accuracy:", accuracy)


# Save the model to a file
joblib.dump(clf, "svm_model.joblib")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import time

# Load the dataset
data = pd.read_csv('/content/public_data_labeled 2.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['full_text'], data['label'], random_state=42)

# Convert the tweets to a matrix of token counts using CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# Transform the matrix of token counts into a normalized tf-idf representation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Train a Naive Bayes classifier on the training set
clf = MultinomialNB()

start = time.time()
clf.fit(X_train_tfidf, y_train)
end = time.time()

# Test the classifier on the testing set
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print('Classification report:\n', report)
print('Accuracy:', accuracy)
print('Time elapsed:', end - start, 'seconds')


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import time

# Load the data
df = pd.read_csv('/content/public_data_labeled 2.csv')

# Load the CountVectorizer object used to transform the training data
vectorizer = CountVectorizer()

# Transform the text data into a matrix of token counts
X = vectorizer.fit_transform(df['full_text'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, random_state=42)

# Define the Naive Bayes classifier
clf = MultinomialNB()

# Define the feature selector using the GeneticSelectionCV class
selector = GeneticSelectionCV(clf,
                              cv=5,
                              verbose=1,
                              scoring="accuracy",
                              n_population=15,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=4,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=2,
                              caching=True,
                              n_jobs=-1)

# Select the best features using the feature selector on the training set
start_time = time.time()
selector.fit(X_train, y_train)
elapsed_time = time.time() - start_time

# Transform the training and testing sets using the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Train the Naive Bayes classifier on the selected features
start_time = time.time()
clf.fit(X_train_selected, y_train)
elapsed_time += time.time() - start_time

# Make predictions on the testing set
y_pred = clf.predict(X_test_selected)

# Calculate accuracy  of naivebayes with genetic algorithm
accuracy = accuracy_score(y_test, y_pred)

# Get the precision, recall, and F1-score for each class
report = classification_report(y_test, y_pred)

# Print the accuracy, precision, recall, F1-score, and elapsed time
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Elapsed Time:", elapsed_time)

# Save the model
import joblib
joblib.dump(clf, "naivebayes_genetic_algorithm.joblib")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split
import time

# Load the data
data = pd.read_csv('/content/public_data_labeled 2.csv')

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['full_text'], data['label'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Train a logistic regression classifier
clf = LogisticRegression()
start_time = time.time()
clf.fit(X_train_vect, y_train)
elapsed_time = time.time() - start_time

# Make predictions on the test set
y_pred = clf.predict(X_test_vect)

# Calculate accuracy, precision and recall of the model
accuracy = accuracy_score(y_test, y_pred)


print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred))
print('Accuracy:', accuracy)

print('Elapsed time:', elapsed_time)
joblib.dump(clf, 'logistic_regression_model.pkl')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from genetic_selection import GeneticSelectionCV
import pandas as pd
import time
import joblib

# Load the data
df = pd.read_csv('/content/public_data_labeled 2.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['full_text'], df['label'], test_size=0.2)

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer to the training data and transform the data
X_train_counts = vectorizer.fit_transform(X_train)

# Create a logistic regression model with max_iter = 5000
clf = LogisticRegression(max_iter=5000)

# Create a GeneticSelectionCV object
selector = GeneticSelectionCV(clf,
                              cv=5,
                              verbose=1,
                              scoring="accuracy",
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=5,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=5,
                              caching=True,
                              n_jobs=-1)

# Record the start time
start_time = time.time()

# Fit the feature selector to the training data and labels
selector.fit(X_train_counts, y_train)

# Transform the training data and test data using the selected features
X_train_selected = selector.transform(X_train_counts)
X_test_selected = selector.transform(vectorizer.transform(X_test))

# Fit the logistic regression model to the selected features
clf.fit(X_train_selected, y_train)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Save the trained model
joblib.dump(clf, "lr_genetic.pkl")

# Print the accuracy and elapsed time
print("Accuracy:", clf.score(X_test_selected, y_test))
print("Elapsed time:", elapsed_time)
